In [1]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa import Agent
from mesa.visualization import SolaraViz, make_space_component

<IPython.core.display.Javascript object>

In [2]:
class UserAgent(Agent):
    def __init__(self, model):
        super().__init__(model)  # Mandatory initialization
        self.role = "Customer"

    def step(self):
        # Example behavior: Move randomly and book tickets
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

        # Simulate booking a ticket
        cinema = self.random.choice([agent for agent in self.model.agents if isinstance(agent, CinemaAgent)])
        if cinema:
            seat_type = self.random.choice(["lower_hall", "upper_gallery", "vip"])
            self.model.booking_system.book_ticket(self, cinema, seat_type)

        # Simulate canceling a ticket (random chance)
        if self.random.random() < 0.1:  # 10% chance to cancel
            self.model.booking_system.cancel_booking(self, cinema)

In [3]:
class BookingSystemAgent(Agent):
    def __init__(self, model):
        super().__init__(model)  # Mandatory initialization
        self.bookings = []  # List to store all bookings

    def book_ticket(self, user, cinema, seat_type):
        # Calculate price based on seat type
        if seat_type == "lower_hall":
            price = 10  # Example price for lower hall
        elif seat_type == "upper_gallery":
            price = 12  # Example price for upper gallery
        elif seat_type == "vip":
            price = 15  # Example price for VIP
        else:
            raise ValueError("Invalid seat type")

        # Check seat availability
        if cinema.available_seats > 0:
            cinema.available_seats -= 1
            booking = {
                "user_id": user.unique_id,
                "cinema_id": cinema.unique_id,
                "seat_type": seat_type,
                "price": price,
            }
            self.bookings.append(booking)
            print(f"Booking successful! User {user.unique_id} booked a {seat_type} seat at {cinema.location} for £{price}.")
        else:
            print(f"No seats available at {cinema.location}.")

    def cancel_booking(self, user, cinema):
        # Find and remove the booking
        for booking in self.bookings:
            if booking["user_id"] == user.unique_id and booking["cinema_id"] == cinema.unique_id:
                self.bookings.remove(booking)
                cinema.available_seats += 1
                print(f"Booking canceled for User {user.unique_id} at {cinema.location}.")
                return
        print(f"No booking found for User {user.unique_id} at {cinema.location}.")

In [4]:
class CinemaAgent(Agent):
    def __init__(self, model, location):
        super().__init__(model)  # Mandatory initialization
        self.location = location
        self.available_seats = 100  # Example: 100 seats per cinema

    def book_seat(self, user):
        if self.available_seats > 0:
            self.available_seats -= 1
            print(f"User {user.unique_id} booked a seat at {self.location}. Seats left: {self.available_seats}")
        else:
            print(f"No seats available at {self.location}.")

In [5]:
class Model(Model):
    def __init__(self, num_users, num_cinemas, seed=None):
        super().__init__(seed=seed)  # Mandatory initialization
        self.num_users = num_users
        self.num_cinemas = num_cinemas
        self.grid = MultiGrid(10, 10, True)  # 10x10 grid
        self.agents = self._all_agents  # Access agents via self.agents
        # Create Booking System Agent
        self.booking_system = BookingSystemAgent(self)
        self.grid.place_agent(self.booking_system, (0, 0))  # Place at a fixed position

        # Create User Agents
        for _ in range(self.num_users):
            user = UserAgent(self)
            self.grid.place_agent(user, (self.random.randrange(10), self.random.randrange(10)))

        # Create Cinema Agents
        for _ in range(self.num_cinemas):
            cinema = CinemaAgent(self, location=f"City {self.random.randrange(4)}")
            self.grid.place_agent(cinema, (self.random.randrange(10), self.random.randrange(10)))

        # Data collection
        self.datacollector = DataCollector(
            model_reporters={"Total_Bookings": lambda m: sum(1 for agent in m.agents if isinstance(agent, UserAgent))},
            agent_reporters={"Role": lambda a: a.role if hasattr(a, "role") else None}
        )

    def step(self):
        self.agents.shuffle_do("step")  # Shuffle and activate agents
        self.datacollector.collect(self)  # Collect data at each step

In [6]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle", "Filled": "true", "Layer": 0, "r": 0.5}
    if isinstance(agent, UserAgent):
        portrayal["Color"] = "blue"
    elif isinstance(agent, CinemaAgent):
        portrayal["Color"] = "red"
    return portrayal

# Create the visualization
model_params = {"num_users": 10, "num_cinemas": 2}
components = [make_space_component(agent_portrayal)]
SolaraViz(Model, model_params, components=components)

Component react.component(mesa.visualization.solara_viz.SolaraViz) raised exception TypeError("SolaraViz() got multiple values for argument 'components'")
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\reacton\core.py", line 1702, in _render
    root_element = el.component.f(*el.args, **el.kwargs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\mesa\mesa_logging.py", line 132, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
TypeError: SolaraViz() got multiple values for argument 'components'


Cannot show ipywidgets in text

In [7]:
if __name__ == "__main__":
    model = Model(num_users=10, num_cinemas=2)
    for _ in range(10):  # Run for 10 steps
        model.step()

    # Print collected data
    print(model.datacollector.get_model_vars_dataframe())
    print(model.datacollector.get_agent_vars_dataframe())

AttributeError: You are trying to set model.agents. In Mesa 3.0 and higher, this attribute is used by Mesa itself, so you cannot use it directly anymore.Please adjust your code to use a different attribute name for custom agent storage.